<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Interacting with Blobs in Storage

## Azure Actions Covered

* Copying files/blobs to blob storage
* Listing files/blobs in blob storage
* Downloading files/blobs from blob storage
* Deleting files/blobs from blob storage

In this lecture, we're going to take a look at how to interact with blobs in storage containers via the Python SDK.

To begin, we import our usual libraries as well as any useful environment variables (e.g. `AZURE_SUBSCRIPTION_ID`). We also need to import the `BlobClient` and `BlobServiceClient`.

In [1]:
# For generating storage account name
import random

from azure.identity import AzureCliCredential
from azure.mgmt.resource import ResourceManagementClient
# New import
from azure.mgmt.storage import StorageManagementClient
from azure.storage.blob import BlobClient, BlobServiceClient

from settings import AZURE_SUBSCRIPTION_ID

Instantiate your Azure credential as well as a `StorageManagementClient`.

In [2]:
credential = AzureCliCredential()
storage_client = StorageManagementClient(credential, AZURE_SUBSCRIPTION_ID)

Let's list our storage accounts so we know which endpoints to use to upload files/blobs.

In [3]:
for account in storage_client.storage_accounts.list():
    print(f'{account.name}: {account.primary_endpoints.blob}')

benbstorage1234: https://benbstorage1234.blob.core.windows.net/


We can use our storage client to create a blob container under our storage account using `blob_containers.create()`. This will take the following parameters:
* `resource_group_name` - Name of resource group where blob container will be created
* `account_name` - Name of the storage account where blob container will be created
* `container_name` - Name for the new blob container
* `blob_container` - Parameters for blob container creation. For full list of parameters, see the [BlobContainer class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-storage/azure.mgmt.storage.v2022_09_01.models.blobcontainer?view=azure-python)

In [ ]:
container = storage_client.blob_containers.create(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    account_name='benbstorage1234',
    container_name='bens-blob-container',
    blob_container={}
)

Let's validate that our container was created.

In [ ]:
container.name

To interact with individual blobs in storage, we need to instantiate a `BlobClient`.

In [6]:
blob_client = BlobClient(
    account_url='https://benbstorage1234.blob.core.windows.net/',
    container_name='bens-blob-container',
    blob_name='sample-blob-1.txt',
    credential=credential
)

Let's use the context manager to upload the test file we created via the command line.

In [7]:
with open('./test_file.txt', 'rb') as data:
    blob_client.upload_blob(data)

Although you can create individual `BlobClient` objects, you can also `BlobServiceClient` to interact with and configure account properties as well as interact with storage containers. You can also use it to get the appropriate client objects for blob entities

In [8]:
blob_service_client = BlobServiceClient(
    account_url='https://benbstorage1234.blob.core.windows.net/',
    credential=credential
)

For example, we can pull by container name to get the storage container client.

In [9]:
container_client = blob_service_client.get_container_client(container='bens-blob-container')

Now that we have the container client, we can list the blobs in the container.

In [10]:
for blob in container_client.list_blob_names():
    print(blob)

sample-blob-1.txt


Similarly, we can get a `BlobClient` object via the `BlobServiceClient`.

In [11]:
download_blob_client = blob_service_client.get_blob_client(
    container='bens-blob-container',
    blob='sample-blob-1.txt'
)

Now let's use the `BlobClient` to download the text blob we uploaded and give it a new name.

In [12]:
with open('python-test-file.txt', 'wb') as data:
    download_stream = download_blob_client.download_blob()
    data.write(download_stream.readall())

Once we've downloaded the blob, we can delete it from our storage container as well.

In [13]:
blob_client.delete_blob()